In [1]:
!pip install -q openai=="1.35.7" "json5==0.9.25"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
!pip install -U qwen-agent

In [3]:
from typing_extensions import ParamSpecKwargs
import pprint
import urllib.parse
import json5
import openai
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

# Add a custom tool named `my_image_gen`.

In [4]:
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    description = 'AI painting (image generation) service, input text description, and return the image URL drawn based on text information.'
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': 'Detailed description of the desired image content, in English',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


# Configure the LLM

In [5]:
openai.api_key = 'Your Secret Key'

llm_cfg = {
    # Use the model service provided by OpenAI:
    'model': 'gpt-4',
    'model_server': 'https://api.openai.com/v1',
    'api_key': 'Your Secret Key',  # You can set the API key directly here or via environment variable

    # (Optional) LLM hyperparameters for generation:
    'generate_cfg': {
        'top_p': 0.8
    }
}

# Create an agent

In [6]:
# Create an agent. Here we use the `Assistant` agent as an example, which is capable of using tools and reading files.
system_instruction = '''You are a helpful assistant.
After receiving the user's request, you should:
- first draw an image and obtain the image url,
- then run code `request.get(image_url)` to download the image,
- and finally select an image operation from the given document to process the image.
Please show the image using `plt.show()`.'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` is a built-in tool for executing code.
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                )


In [8]:
# Run the agent as a chatbot.
messages = []  # This stores the chat history.
while True:
    query = input('user query: ')
    if query=='quit':
      break
    messages.append({'role': 'user', 'content': query})
    response = []
    for response in bot.run(messages=messages):
        pass
    print(response)
    messages.extend(response)


user query: create an image white house
[{'role': 'assistant', 'content': "Sure, let's use the image generation tool to create an image of the White House."}, {'role': 'assistant', 'content': '', 'function_call': {'name': 'my_image_gen', 'arguments': '{"prompt": "White House"}'}}, {'role': 'function', 'content': '{image_url: "https://image.pollinations.ai/prompt/White%20House"}', 'name': 'my_image_gen'}, {'role': 'assistant', 'content': "Great, the image has been generated. Now let's download the image using the Python code."}, {'role': 'assistant', 'content': '', 'function_call': {'name': 'code_interpreter', 'arguments': '{"code": "import requests\\nimage_url = \'https://image.pollinations.ai/prompt/White%20House\'\\nimage = requests.get(image_url)\\nimage"}'}}, {'role': 'function', 'content': 'execute_result:\n\n```\n<Response [200]>\n```', 'name': 'code_interpreter'}, {'role': 'assistant', 'content': "Now that we have downloaded the image, let's use the matplotlib library to display